curl -X GET "https://your-development-store.myshopify.com/admin/api/2023-07/products.json" \
-H "X-Shopify-Access-Token: {access_token}"

In [5]:
from bs4 import BeautifulSoup
import requests
import os
from dotenv import load_dotenv
import time
import pandas as pd
pd.set_option("display.max_colwidth", 1000)
load_dotenv()

shop_url = "https://0349d9.myshopify.com"
api_version = "2023-07"

def get_all_products(shop_url, api_version):
    all_products = []
    url = f"{shop_url}/admin/api/{api_version}/products.json"
    headers = {"X-Shopify-Access-Token": os.getenv("SHOPIFY_API_KEY")}
    params = {"limit": 250}
    response = requests.get(url, headers=headers, params=params)
    all_products.extend(response.json()["products"])
    try:
        while response.links["next"]:
            response = requests.get(response.links["next"]["url"], headers=headers)
            all_products.extend(response.json()["products"])
            time.sleep(2)
    except KeyError:
        return all_products
    
def clean_html_tags(row):
    soup = BeautifulSoup(row["body_html"], "html.parser")
    text = soup.get_text()
    row["body_html"] = text
    return row

def get_img_src(row):
    all_images = []
    for image in row["images"]:
        all_images.append(image["src"])
    row["images_list"] = all_images
    return row

def create_expandend_description(row):
    if row["body_html"] == "" and row["tags"] == "":
        row["expanded_description"] = row["title"]
    elif row["body_html"] == "" and row["tags"] != "":
        row["expanded_description"] = "Title: " + row['title'] + " Tags: " + row['tags']
    elif row["body_html"] != "" and row["tags"] == "":
        row["expanded_description"] = "Title: " + row['title'] + " Description: " +row["body_html"]
    else:
        row["expanded_description"] = "Title: " + row['title'] + " Description: " +row["body_html"] + " Tags: " + row['tags']
    return row

def df_preprocessing(df):
    df = df[df["status"] == "active"]
    df.fillna("", inplace=True)
    df = df.apply(lambda row: get_img_src(row), axis=1)
    df = df.apply(lambda row: create_expandend_description(row), axis=1)
    df = df.apply(lambda row: clean_html_tags(row), axis=1)
    df = df.rename(columns={"body_html": "description"})
    df = df[["id", "title", "handle","description", "expanded_description", "images_list"]]
    return df

all_products = get_all_products(shop_url, api_version)
product_df = pd.DataFrame(all_products)
cleaned_df = df_preprocessing(product_df)


cleaned_df.to_csv("products.csv", index=False)
cleaned_products_json = cleaned_df.to_json(orient="records")
with open("products.json", "w") as f:
    f.write(cleaned_products_json)

/tmp/ipykernel_7147/3647438458.py:29: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row["body_html"], "html.parser")


In [28]:
from langchain.document_loaders.json_loader import JSONLoader
# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["id"] = record.get("id")
    metadata["title"] = record.get("title")
    metadata["tags"] = record.get("tags")
    metadata["images_list"] = record.get("images_list")
    metadata["handle"] = record.get("handle")
    return metadata


loader = JSONLoader(
    file_path='./products.json',
    jq_schema='.[]',
    content_key="expanded_description",
    metadata_func=metadata_func
)

documents = loader.load()


In [31]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0)

def count_tokens(text):
    return splitter.count_tokens(text=text)

summed_tokens = 0
for product in documents:
    summed_tokens += count_tokens(product.page_content)

summed_tokens / 1000 * 0.0001 

0.0003005

In [1]:
from utils import load_vectorstore
from dotenv import load_dotenv
load_dotenv()

vectorstore = load_vectorstore(vectorstore_path="./shopify_langchaintesting_vectorstore", index_name="products")

In [2]:
result = vectorstore.similarity_search("I want a adidas shoe for my child")

In [4]:
for product in result:
    print(product.page_content)
    print(product.metadata["images_list"])

Title: ADIDAS | KID'S STAN SMITH Description: The Stan Smith owned the tennis court in the '70s. Today it runs the streets with the same clean, classic style. These kids' shoes preserve the iconic look of the original, made in leather with punched 3-Stripes, heel and tongue logos and lightweight step-in cushioning. Tags: adidas, egnition-sample-data, kid
['https://cdn.shopify.com/s/files/1/0772/8542/5497/products/7883dc186e15bf29dad696e1e989e914.jpg?v=1692705055', 'https://cdn.shopify.com/s/files/1/0772/8542/5497/products/8cd561824439482e3cea5ba8e3a6e2f6.jpg?v=1692705056', 'https://cdn.shopify.com/s/files/1/0772/8542/5497/products/2e1f72987692d2dcc3c02be2f194d6c5.jpg?v=1692705056', 'https://cdn.shopify.com/s/files/1/0772/8542/5497/products/6216e82660d881e6f2b0e46dc3f8844a.jpg?v=1692705056', 'https://cdn.shopify.com/s/files/1/0772/8542/5497/products/e5247cc373e3b61f18013282a6d9c3c0.jpg?v=1692705056']
Title: NIKE | TODDLER ROSHE ONE Description: The Nike Roshe One Infant Shoe offers brea

In [2]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()
header = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + os.getenv("SHOPIFY_STATIC_TOKEN")
}
response = requests.get("https://shopify-test-14xv.onrender.com/product_search/", params={"query": "I want a adidas shoe for my child", "k": 2}, headers=header)
response.json()

[{'page_content': "Title: ADIDAS | KID'S STAN SMITH Description: The Stan Smith owned the tennis court in the '70s. Today it runs the streets with the same clean, classic style. These kids' shoes preserve the iconic look of the original, made in leather with punched 3-Stripes, heel and tongue logos and lightweight step-in cushioning. Tags: adidas, egnition-sample-data, kid",
  'metadata': {'source': '/home/leon/code/shopify_langchain_testing/products.json',
   'seq_num': 3,
   'id': 8497889673561,
   'title': "ADIDAS | KID'S STAN SMITH",
   'tags': None,
   'images_list': ['https://cdn.shopify.com/s/files/1/0772/8542/5497/products/7883dc186e15bf29dad696e1e989e914.jpg?v=1692705055',
    'https://cdn.shopify.com/s/files/1/0772/8542/5497/products/8cd561824439482e3cea5ba8e3a6e2f6.jpg?v=1692705056',
    'https://cdn.shopify.com/s/files/1/0772/8542/5497/products/2e1f72987692d2dcc3c02be2f194d6c5.jpg?v=1692705056',
    'https://cdn.shopify.com/s/files/1/0772/8542/5497/products/6216e82660d881e6

In [6]:
import os
import base64
token_bytes = os.urandom(24)
safe_token = base64.urlsafe_b64encode(token_bytes).decode('utf-8')
print(safe_token)
print("Token length:", len(safe_token))

18mIeyZHogqcYV--EiSDUWpc9Uscjh02
Token length: 32


In [3]:
response.json()

[{'page_content': "Title: ADIDAS | KID'S STAN SMITH Description: The Stan Smith owned the tennis court in the '70s. Today it runs the streets with the same clean, classic style. These kids' shoes preserve the iconic look of the original, made in leather with punched 3-Stripes, heel and tongue logos and lightweight step-in cushioning. Tags: adidas, egnition-sample-data, kid",
  'metadata': {'source': '/home/leon/code/shopify_langchain_testing/products.json',
   'seq_num': 3,
   'id': 8497889673561,
   'title': "ADIDAS | KID'S STAN SMITH",
   'tags': None,
   'images_list': ['https://cdn.shopify.com/s/files/1/0772/8542/5497/products/7883dc186e15bf29dad696e1e989e914.jpg?v=1692705055',
    'https://cdn.shopify.com/s/files/1/0772/8542/5497/products/8cd561824439482e3cea5ba8e3a6e2f6.jpg?v=1692705056',
    'https://cdn.shopify.com/s/files/1/0772/8542/5497/products/2e1f72987692d2dcc3c02be2f194d6c5.jpg?v=1692705056',
    'https://cdn.shopify.com/s/files/1/0772/8542/5497/products/6216e82660d881e6

In [5]:
"https://0349d9.myshopify.com/products/" + response.json()[0]["metadata"]["handle"]

'https://0349d9.myshopify.com/products/adidas-kids-stan-smith'

In [4]:
from create_vectorstore import create_vectorstore, save_vectorstore
from langchain.schema.document import Document

doc = """Send returns exclusively to the following return address:

Do you want to return an unwanted item?

We understand that sometimes an item is simply not right for you and you want your money back. As long as the item is still in its original condition, we accept returns, subject to the rules listed below, which also include fair usage rules.

If you return an item within 30 days of the delivery date or the date your package was ready for pickup, we will initiate a refund to your original payment method.

If you return an item between 31 and 45 days from the delivery date or the date your package was ready for pickup, we will issue you a WAAGEMANN gift voucher for the amount you paid. To learn more about our new return policy, please click here.

If you want to return an item after the maximum return period, you can hope for goodwill on our part, as we aim for complete customer satisfaction.

We do our best to process your refund within 14 days of receiving your returned item.

If you prefer a refund within the above-mentioned periods but can't return the items to us for some reason, please contact us. However, a refund in this case is at our discretion.
And then?

After the respective return periods, we no longer accept returns for unwanted items. If you attempt to return something, we may send the item back to your default delivery address and ask you to cover the shipping costs.
What happens to my refund if I paid with Klarna?

If you have placed an order with Klarna, the payable amount is due within 14 days of placing the order. If you return items within 30 days, we will issue a full refund for you.

If you choose to return items between 31 and 45 days from the delivery date or the date your package was ready for pickup, we will instead issue a gift voucher, and your payment obligation with Klarna remains. WAAGEMANN assumes no responsibility for these claims or any resulting fees, so please make sure you have settled the full amount!
Original Condition

We strive for complete customer satisfaction, and each return case is an individual case for us and is treated as such. In almost all cases, we always find a way to satisfy our customers.

In principle, items that have been briefly used can also be returned, as long as they do not contradict the conditions listed below and have not suffered damage or defects due to your actions.

Hygiene and the safety of our customers are super important to us, so certain items cannot be returned for a refund. These include:

    Face + Body items if they have been opened or used, or if the protective seal has been removed
    Underwear if the hygiene seal is no longer intact or if any tags are damaged
    Swimwear if the hygiene seal is no longer intact or if any tags are damaged
    Piercings and jewelry if the hygiene seal is no longer intact or damaged

Your statutory rights are not affected.

Of course, it's okay to try on an item as you would in a store, but please do not wear items you want to return. If an item is returned to us that is damaged, worn, or otherwise in an unacceptable condition, we will not be able to issue you a refund, and we may send the item back to you (and ask you to cover the shipping costs). All returns are checked upon arrival.

All returned items should be sent back to us in their original condition and original packaging, including the price tag (e.g., shoes should be returned in their original shoebox).
Responsibility

Please make sure your items are securely packed and cannot be damaged in transit.

Since the package is your responsibility until it reaches us, please obtain a shipping receipt!

We assume no responsibility for items that are accidentally sent back to us (it can happen!). If we can find the items (which is not always possible) and you want them back, we may ask you to cover the shipping costs.
Return Costs

The buyer and customer generally bear the return costs.
Package cannot be delivered or is refused

If the package cannot be delivered to the buyer, or it is not accepted by the seller, we reserve the right to retain a delivery & service fee of €10 from the refund amount.
Fair Use

If we notice an unusual pattern in your returns that seems strange to us, and where we suspect, for example, that someone is wearing the ordered items and then returning them, or that someone is ordering and returning much more than even the most loyal WAAGEMANN customers would order, or that the returned items do not match what you ordered—then we may have to deactivate the respective account and any associated accounts. If this happens and you think we've made a mistake, please contact our customer service, and we'll be happy to discuss it with you.

If your customer account has been deactivated and you need to make a valid return to us, our customer service can issue you a return label, but you must bear the costs for returning the goods.

Please note that we reserve the right to take legal action against you if the items you are returning do not match those you ordered.
Can I exchange my item?

We generally offer an exchange, but you can still return your order to us. If you want to buy a different size or color, return the unwanted item, postage paid, and place a new order.

"""

refund = Document(page_content=doc)

In [6]:
from langchain.embeddings import OpenAIEmbeddings
vec = create_vectorstore(documents=[refund], embeddings=OpenAIEmbeddings())
save_vectorstore(vec, save_path="./shopify_langchaintesting_vectorstore", index_name="refund")

Vectorstore saved to:  ./shopify_langchaintesting_vectorstore
